In [3]:
import simpy
import random
import numpy as np

# Simulation parameters
RANDOM_SEED = 42
SIM_TIME = 12 * 60  # 12 hours in minutes
ARRIVAL_RATE = 5  # Patients arrive every 5 minutes (Poisson process)
NUM_DOCTORS = 3  # Number of doctors available
NUM_NURSES = 2  # Number of nurses for triage
TRIAGE_TIME = (2, 6)  # Triage takes between 2-6 minutes
TREATMENT_TIME = (10, 30)  # Treatment takes between 10-30 minutes

# Metrics storage
waiting_times = []
treatment_times = []
total_times = []

class EmergencyDepartment:
    def __init__(self, env, num_doctors, num_nurses):
        self.env = env
        self.doctor = simpy.Resource(env, num_doctors)  # Doctors as limited resources
        self.nurse = simpy.Resource(env, num_nurses)  # Nurses for triage

    def triage(self, patient):
        """ Simulate triage process (handled by nurses). """
        triage_time = random.randint(*TRIAGE_TIME)
        yield self.env.timeout(triage_time)
        return triage_time

    def treat(self, patient):
        """ Simulate treatment process (handled by doctors). """
        treatment_time = random.randint(*TREATMENT_TIME)
        yield self.env.timeout(treatment_time)
        return treatment_time

def patient(env, name, ed):
    """ Models a single patient going through ED processes. """
    arrival_time = env.now

    # Step 1: Triage
    with ed.nurse.request() as req:
        yield req
        triage_time = yield env.process(ed.triage(name))

    # Step 2: Wait for doctor and treatment
    with ed.doctor.request() as req:
        wait_time = env.now - arrival_time
        waiting_times.append(wait_time)
        yield req
        treatment_time = yield env.process(ed.treat(name))
        treatment_times.append(treatment_time)

    # Step 3: Discharge
    total_time = env.now - arrival_time
    total_times.append(total_time)
    print(f"Patient {name} completed ED visit in {total_time:.2f} minutes (Wait: {wait_time:.2f}, Triage: {triage_time:.2f}, Treatment: {treatment_time:.2f})")

def patient_generator(env, ed):
    """ Generates patients arriving at the ED following a Poisson process. """
    patient_id = 0
    while True:
        interarrival_time = random.expovariate(1.0 / ARRIVAL_RATE)  # Poisson process
        yield env.timeout(interarrival_time)
        patient_id += 1
        env.process(patient(env, f"Patient-{patient_id}", ed))

# Run simulation
random.seed(RANDOM_SEED)
env = simpy.Environment()
ed = EmergencyDepartment(env, NUM_DOCTORS, NUM_NURSES)
env.process(patient_generator(env, ed))
env.run(until=SIM_TIME)

# Statistical analysis
print("\n--- Simulation Results ---")
print(f"Average Waiting Time: {np.mean(waiting_times):.2f} minutes")
print(f"Average Treatment Time: {np.mean(treatment_times):.2f} minutes")
print(f"Average Total Time in ED: {np.mean(total_times):.2f} minutes")


Patient Patient-3 completed ED visit in 26.60 minutes (Wait: 3.60, Triage: 2.00, Treatment: 23.00)
Patient Patient-2 completed ED visit in 30.00 minutes (Wait: 3.00, Triage: 3.00, Treatment: 27.00)
Patient Patient-1 completed ED visit in 32.00 minutes (Wait: 4.00, Triage: 4.00, Treatment: 28.00)
Patient Patient-6 completed ED visit in 35.41 minutes (Wait: 6.60, Triage: 6.00, Treatment: 13.00)
Patient Patient-5 completed ED visit in 37.77 minutes (Wait: 6.00, Triage: 6.00, Treatment: 16.00)
Patient Patient-4 completed ED visit in 42.93 minutes (Wait: 2.00, Triage: 2.00, Treatment: 23.00)
Patient Patient-7 completed ED visit in 42.80 minutes (Wait: 6.16, Triage: 6.00, Treatment: 12.00)
Patient Patient-8 completed ED visit in 52.82 minutes (Wait: 5.89, Triage: 5.00, Treatment: 22.00)
Patient Patient-9 completed ED visit in 57.10 minutes (Wait: 4.33, Triage: 2.00, Treatment: 24.00)
Patient Patient-11 completed ED visit in 49.54 minutes (Wait: 3.00, Triage: 3.00, Treatment: 11.00)
Patient P

In [4]:
import simpy
import random
import numpy as np
from heapq import heappop, heappush

# Simulation parameters
RANDOM_SEED = 42
SIM_TIME = 12 * 60  # 12 hours in minutes
ARRIVAL_RATE = 5  # Patients arrive every 5 minutes (Poisson process)
NUM_DOCTORS = 3  # Number of doctors
NUM_NURSES = 2  # Number of nurses for triage
TRIAGE_TIME = (2, 6)  # Triage time range in minutes
TREATMENT_TIME = {1: (10, 20), 2: (15, 25), 3: (20, 30)}  # Treatment time based on priority

# Metrics storage
waiting_times = []
treatment_times = []
total_times = []

class EmergencyDepartment:
    def __init__(self, env, num_doctors, num_nurses):
        self.env = env
        self.doctor = simpy.PriorityResource(env, num_doctors)  # Priority resource for doctors
        self.nurse = simpy.Resource(env, num_nurses)  # Regular resource for triage
        self.waiting_queue = []  # Priority queue for patients waiting for doctors

    def triage(self, patient):
        """ Simulate triage process, assigns a priority. """
        triage_time = random.randint(*TRIAGE_TIME)
        yield self.env.timeout(triage_time)
        priority = random.choices([1, 2, 3], weights=[0.2, 0.5, 0.3])[0]  # 20% Critical, 50% Urgent, 30% Non-Urgent
        return triage_time, priority

    def treat(self, patient, priority):
        """ Simulate treatment process based on priority. """
        treatment_time = random.randint(*TREATMENT_TIME[priority])
        yield self.env.timeout(treatment_time)
        return treatment_time

def patient(env, name, ed):
    """ Models a single patient going through ED processes. """
    arrival_time = env.now

    # Step 1: Triage
    with ed.nurse.request() as req:
        yield req
        triage_time, priority = yield env.process(ed.triage(name))

    # Step 2: Wait for doctor based on priority
    with ed.doctor.request(priority=priority) as req:
        heappush(ed.waiting_queue, (priority, env.now, name))
        yield req
        _, wait_start, _ = heappop(ed.waiting_queue)
        wait_time = env.now - wait_start
        waiting_times.append(wait_time)
        
        # Step 3: Treatment
        treatment_time = yield env.process(ed.treat(name, priority))
        treatment_times.append(treatment_time)

    # Step 4: Discharge
    total_time = env.now - arrival_time
    total_times.append(total_time)
    print(f"Patient {name} (Priority {priority}) completed ED visit in {total_time:.2f} minutes (Wait: {wait_time:.2f}, Triage: {triage_time:.2f}, Treatment: {treatment_time:.2f})")

def patient_generator(env, ed):
    """ Generates patients arriving at the ED following a Poisson process. """
    patient_id = 0
    while True:
        interarrival_time = random.expovariate(1.0 / ARRIVAL_RATE)  # Poisson arrivals
        yield env.timeout(interarrival_time)
        patient_id += 1
        env.process(patient(env, f"Patient-{patient_id}", ed))

# Run simulation
random.seed(RANDOM_SEED)
env = simpy.Environment()
ed = EmergencyDepartment(env, NUM_DOCTORS, NUM_NURSES)
env.process(patient_generator(env, ed))
env.run(until=SIM_TIME)

# Statistical analysis
print("\n--- Simulation Results ---")
print(f"Average Waiting Time: {np.mean(waiting_times):.2f} minutes")
print(f"Average Treatment Time: {np.mean(treatment_times):.2f} minutes")
print(f"Average Total Time in ED: {np.mean(total_times):.2f} minutes")


Patient Patient-3 (Priority 1) completed ED visit in 16.60 minutes (Wait: 0.00, Triage: 2.00, Treatment: 13.00)
Patient Patient-1 (Priority 2) completed ED visit in 19.00 minutes (Wait: 0.00, Triage: 4.00, Treatment: 15.00)
Patient Patient-2 (Priority 2) completed ED visit in 26.00 minutes (Wait: 0.00, Triage: 3.00, Treatment: 23.00)
Patient Patient-7 (Priority 1) completed ED visit in 18.17 minutes (Wait: 0.17, Triage: 2.00, Treatment: 16.00)
Patient Patient-4 (Priority 2) completed ED visit in 30.80 minutes (Wait: 4.80, Triage: 6.00, Treatment: 20.00)
Patient Patient-8 (Priority 1) completed ED visit in 20.73 minutes (Wait: 2.73, Triage: 3.00, Treatment: 15.00)
Patient Patient-9 (Priority 2) completed ED visit in 32.10 minutes (Wait: 8.10, Triage: 4.00, Treatment: 20.00)
Patient Patient-10 (Priority 2) completed ED visit in 24.34 minutes (Wait: 1.34, Triage: 2.00, Treatment: 21.00)
Patient Patient-5 (Priority 3) completed ED visit in 56.48 minutes (Wait: 26.48, Triage: 3.00, Treatmen

In [5]:
import streamlit as st
import simpy
import random
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

# Simulation parameters
RANDOM_SEED = 42
SIM_TIME = 12 * 60  # 12 hours
ARRIVAL_RATE = 5  # Patients arrive every 5 minutes
NUM_DOCTORS = 3  
NUM_NURSES = 2  
TRIAGE_TIME = (2, 6)  
TREATMENT_TIME = {1: (10, 20), 2: (15, 25), 3: (20, 30)}  

# Data storage
waiting_times = []
treatment_times = []
total_times = []
triage_counts = {1: 0, 2: 0, 3: 0}
live_patient_queue = []

# Streamlit UI
st.title("🏥 Emergency Department Simulation Dashboard")
status_placeholder = st.empty()
chart_placeholder = st.empty()

class EmergencyDepartment:
    def __init__(self, env, num_doctors, num_nurses):
        self.env = env
        self.doctor = simpy.PriorityResource(env, num_doctors)  
        self.nurse = simpy.Resource(env, num_nurses)  

    def triage(self, patient):
        triage_time = random.randint(*TRIAGE_TIME)
        yield self.env.timeout(triage_time)
        priority = random.choices([1, 2, 3], weights=[0.2, 0.5, 0.3])[0]  
        triage_counts[priority] += 1
        return triage_time, priority

    def treat(self, patient, priority):
        treatment_time = random.randint(*TREATMENT_TIME[priority])
        yield self.env.timeout(treatment_time)
        return treatment_time

def patient(env, name, ed):
    arrival_time = env.now

    with ed.nurse.request() as req:
        yield req
        triage_time, priority = yield env.process(ed.triage(name))
    
    with ed.doctor.request(priority=priority) as req:
        live_patient_queue.append((name, priority))
        yield req
        wait_time = env.now - arrival_time
        waiting_times.append(wait_time)
        live_patient_queue.remove((name, priority))

        treatment_time = yield env.process(ed.treat(name, priority))
        treatment_times.append(treatment_time)

    total_time = env.now - arrival_time
    total_times.append(total_time)

def patient_generator(env, ed):
    patient_id = 0
    while True:
        yield env.timeout(random.expovariate(1.0 / ARRIVAL_RATE))
        patient_id += 1
        env.process(patient(env, f"Patient-{patient_id}", ed))

# Streamlit simulation control
def run_simulation():
    random.seed(RANDOM_SEED)
    env = simpy.Environment()
    ed = EmergencyDepartment(env, NUM_DOCTORS, NUM_NURSES)
    env.process(patient_generator(env, ed))

    while env.peek() < SIM_TIME:
        env.step()
        time.sleep(0.1)  # Simulate real-time effect

        # Update patient queue & stats
        status_placeholder.markdown(f"### ⏳ Live Patient Queue ({len(live_patient_queue)})")
        queue_df = pd.DataFrame(live_patient_queue, columns=["Patient", "Priority"])
        status_placeholder.dataframe(queue_df)

        # Update triage distribution chart
        triage_df = pd.DataFrame.from_dict(triage_counts, orient='index', columns=['Count'])
        triage_df.index = ["Critical (1)", "Urgent (2)", "Non-Urgent (3)"]

        fig, ax = plt.subplots()
        triage_df.plot(kind='bar', legend=False, ax=ax, color=['red', 'orange', 'green'])
        ax.set_title("Triage Distribution")
        ax.set_ylabel("Number of Patients")
        ax.set_xlabel("Priority Level")
        chart_placeholder.pyplot(fig)

    st.success("Simulation Complete! ✅")

if st.button("🚀 Start Simulation"):
    run_simulation()


2025-04-04 13:58:33.671 
  command:

    streamlit run C:\Users\gangl\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [8]:
streamlit run ed_simulation_dashboard.py

SyntaxError: invalid syntax (2348846883.py, line 1)